# Logistic Regression (MNIST dataset)

## Import Module

In [ ]:
%matplotlib inline
from keras.datasets import mnist
from keras.utils import to_categorical
import math
import numpy as np
import requests
import pandas as pd
import matplotlib.pyplot as plt

## Load Dataset

In [ ]:
((X_train, y_train), (X_test, y_test)) = mnist.load_data()

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

In [ ]:
# X_train : Train dataset(Image feature data) / X_test : Test dataset(Image feature data)
X_train = X_train.reshape(60000, 28*28)
X_test = X_test.reshape(10000, 28*28)

print(X_train.shape, X_test.shape)

In [ ]:
# y_train: Train dataset(label data) / y_test : Test dataset(label data)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(y_train.shape, y_test.shape)

In [ ]:
# Show Image of X_train[0]
plt.matshow(X_train[0].reshape(28, 28))

In [ ]:
# Show result label of X_train[0] (=5)
y_train[0].argmax()

## Sigmoid Function

In [ ]:
# For Calculating Cost (calculate value range from 0 to 1)
def sigmoid(n):
    return 1 / (1 + np.exp(-n))

print(sigmoid(-9))
print(sigmoid(0))
print(sigmoid(+9))

In [ ]:
x = range(-100, 100)
y = [sigmoid(i/10) for i in x]
plt.plot(x, y)

## Gradient Descent

In [ ]:
learning_rate = 1
num_epoch = 100

In [ ]:
# Parameter
w = np.random.uniform(low=-1.0, high=1.0, size=(28*28, 10))
b = np.random.uniform(low=-1.0, high=1.0, size=10)

In [ ]:
for epoch in range(num_epoch):
    # Predict y (=number 0~9)
    y_predict = X_train.dot(w) + b
    y_predict = sigmoid(y_predict)
    
    # Calculating Error
    error = (y_predict.argmax(axis=1) != y_train.argmax(axis=1)).mean()
    
    # Print Error
    if epoch % 10 == 0:
        print("{0:4} error = {1:.5f}".format(epoch, error))
    
    # Gradient Descent for Multiple Variables
    w = w - learning_rate * X_train.T.dot(y_predict - y_train)
    b = b - learning_rate * (y_predict - y_train).mean(axis=0)

## Train

In [ ]:
# Predict using calculated parameter
y_predict = X_train.dot(w) + b
y_predict = sigmoid(y_predict)

In [ ]:
# Make Result Dataframe
actual = y_train.argmax(axis=1)
predict = y_predict.argmax(axis=1)

pd.DataFrame({'actual': actual, 'predict': predict}).head(10)

## Test

In [ ]:
# Predict using calculated parameter
y_predict = X_test.dot(w) + b
y_predict = sigmoid(y_predict)

In [ ]:
# Make Result Dataframe
actual = y_test.argmax(axis=1)
predict = y_predict.argmax(axis=1)

result = pd.DataFrame({'actual': actual, 'predict': predict})
result.head(10)

In [ ]:
# Print Error
plt.matshow(X_test[8].reshape(28, 28))

In [ ]:
# Calculating accuracy
accuracy = []

for i in range(10):
    chunk = result[result["actual"] == i]
    accuracy.append((chunk["actual"] == chunk["predict"]).mean())

In [ ]:
result = pd.DataFrame({'accuracy' : accuracy})
result

## Score

### Make request to azure function for score

In [ ]:
_url = "https://jinheon-azureml-score.azurewebsites.net/api/Logistic-Regression-Score?code=gG5CmUzXsaBY2bCStVdAvltwagL4e7q15zTipWJCDW0xsgMlVUamUg=="

In [ ]:
json = {
    "name": "Jinheon",
    "score": result['accuracy'].mean()
}

In [ ]:
req = requests.request('POST', _url, json = json)

In [ ]:
print(req.json())